In [ ]:
# Install necessary libraries
!pip install transformers torch accelerate

# Import required modules
import json
import os
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset from the uploaded file
dataset_path = "/content/drive/MyDrive/Project-3/dataset.json"  # Adjust the path if necessary
with open(dataset_path, "r") as f:
    dataset = json.load(f)

# Inspect the dataset
print(f"Total conversations: {len(dataset)}")
example_conversation = next(iter(dataset.values()))
print("Sample conversation structure:", example_conversation)


Total conversations: 7168
Sample conversation structure: {'messages': [[{'text': 'Hello', 'timestamp': '2018-05-02T19:38:15Z', 'sender': '720840be-e522-47ba-9e9f-143f66372673'}, {'text': 'How are you doing today?', 'timestamp': '2018-05-02T19:38:20Z', 'sender': '720840be-e522-47ba-9e9f-143f66372673'}], [{'text': 'whats up MD', 'timestamp': '2018-05-02T19:38:29Z', 'sender': '549cc1d1-270e-4a53-b561-133a8d0086b4'}, {'text': 'im doing good', 'timestamp': '2018-05-02T19:38:33Z', 'sender': '549cc1d1-270e-4a53-b561-133a8d0086b4'}, {'text': 'how are you doing?', 'timestamp': '2018-05-02T19:38:35Z', 'sender': '549cc1d1-270e-4a53-b561-133a8d0086b4'}], [{'text': 'Im alright, I just took a nap. But it was one of those naps that doesnt help anything. It just makes everything worse and you question all your life choices', 'timestamp': '2018-05-02T19:39:21Z', 'sender': '720840be-e522-47ba-9e9f-143f66372673'}], [{'text': 'oh wow haha', 'timestamp': '2018-05-02T19:39:58Z', 'sender': '549cc1d1-270e-4a5

In [ ]:
def prepare_training_pairs(data):
    training_pairs = []
    for conv in data.values():
        messages = conv["messages"]
        utterances = [" ".join([u["text"] for u in turn]) for turn in messages]
        for i in range(1, len(utterances)):
            input_text = "<speaker1>" + "<speaker2>".join(utterances[:i])
            response_text = "<speaker2>" + utterances[i]
            training_pairs.append((input_text, response_text))
    return training_pairs

# Prepare training data
training_pairs = prepare_training_pairs(dataset)
print(f"Total training pairs: {len(training_pairs)}")
print("Sample training pair:")
print("Input:", training_pairs[0][0])
print("Response:", training_pairs[0][1])


Total training pairs: 131569
Sample training pair:
Input: <speaker1>Hello How are you doing today?
Response: <speaker2>whats up MD im doing good how are you doing?


In [ ]:
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

class ChitChatDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_length=128):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        input_text, response_text = self.pairs[idx]
        inputs = self.tokenizer(
            input_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt"
        )
        responses = self.tokenizer(
            response_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt"
        )
        return {"input_ids": inputs.input_ids.squeeze(), "labels": responses.input_ids.squeeze()}


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load the pre-trained DialoGPT model and tokenizer
model_name = "microsoft/DialoGPT-medium"
print("Loading pre-trained DialoGPT model...")
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
model.to(device)

Loading pre-trained DialoGPT model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
# Load the pre-trained DialoGPT model and tokenizer
model_name = "microsoft/DialoGPT-medium"
print("Loading pre-trained DialoGPT model...")
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
model.to(device)

Loading pre-trained DialoGPT model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
from transformers import TrainingArguments

# Split data for validation
val_pairs = training_pairs[:len(training_pairs) // 10]  # Use 10% for validation
train_dataset = ChitChatDataset(training_pairs, tokenizer)
val_dataset = ChitChatDataset(val_pairs, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Project-3/chatbot_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_steps=1000,
    save_total_limit=2,
    logging_dir="/content/drive/MyDrive/Project-3/chatbot_logs",
    report_to="none",  # Disable external logging, including W&B
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

# Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


<ipython-input-9-41e1906976cd>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_WANDB_LOG_LEVEL"] = "error"  # Suppress W&B-related warnings

# Fine-tune the model
print("Starting fine-tuning...")
trainer.train()

# Save the fine-tuned model
print("Model fine-tuned and saved to './results'.")
model.save_pretrained("/content/drive/MyDrive/Project-3/chatbot_finalmodel")
tokenizer.save_pretrained("/content/drive/MyDrive/Project-3/chatbot_finaltokenizer")


Starting fine-tuning...


Epoch,Training Loss,Validation Loss
1,1.270100,1.187740
2,1.250000,1.173131
3,1.245200,1.164923


Model fine-tuned and saved to './results'.


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [ ]:
import shutil

# Path to the folder you want to zip
folder_to_zip = "./fine_tuned_dialoGPT"

# Create a zip file
shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)

print(f"Folder compressed as {folder_to_zip}.zip")

Folder compressed as ./fine_tuned_dialoGPT.zip


In [ ]:
# Destination path in Google Drive
drive_destination = "/content/drive/MyDrive/Project-3/fine_tuned_dialoGPT.zip"

# Copy the zip file to Google Drive
shutil.copy('/content/fine_tuned_dialoGPT.zip', drive_destination)

print(f"Zip file stored in Google Drive at {drive_destination}")

Zip file stored in Google Drive at /content/drive/MyDrive/Project-3/fine_tuned_dialoGPT.zip


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Project-3/chatbot_finalmodel/dialoGPT_weights.pth")

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Project-3/chatbot_finalmodel/dialoGPT_weights.pth', map_location="cuda"))

<ipython-input-8-8348e318061e>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/Project-3/chatbot_finalmodel/dialoGP

<All keys matched successfully>

In [ ]:
model.save_pretrained('./DialoGPT')

In [ ]:
# Load the model and tokenizer
from safetensors.torch import load_file
model_name = "microsoft/DialoGPT-medium"  # Replace with your model name
safetensors_path = "/content/DialoGPT/model.safetensors"  # Your safetensors file path

# Load the model's configuration
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the safetensors model weights
state_dict = load_file(safetensors_path)

# Apply the state dict to the model
model.load_state_dict(state_dict,strict=False)

# Save the model as pytorch_model.bin
model.save_pretrained("model")

# Optionally, save the tokenizer as well
# tokenizer.save_pretrained("/path/to/save/model")

In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:Hi!
DialoGPT: Hi! :D
>> User:How r u?
DialoGPT: I'm good, how are you?
>> User:Very good
DialoGPT: That's good!
>> User:How are things?
DialoGPT: They're good!


KeyboardInterrupt: Interrupted by user

In [ ]:
# Example usage:
while True:
  user_input = input("User: ")
  response = generate_response(model, tokenizer, user_input)
  print("Model:", response)

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhprajapat/chatbot-training-dataset")

print("Path to dataset files:", path)

100%|██████████| 17.8k/17.8k [00:00<00:00, 19.2MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/saurabhprajapat/chatbot-training-dataset/versions/1
